In [41]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split


In [42]:
import joblib
import pandas as pd

data_kcat = joblib.load('/Users/pinchichen/2025S lab/AI drug project/Catpred/dataset/kcat/kcat_with_features.joblib')
data_kcat.head()

,Sequence,sequence_source,uniprot,Smiles,value,ec,log10_value,reactant_smiles,product_smiles,log10kcat_max,metabolite_features,protein_features
0,MEPGSKSVSRSDWQPEPHQRPITPLEPGPEKTPIAQPESKTLQGSN...,sabio,Q05469,CCC(=O)OCC(COC(=O)CC)OC(=O)CC.O>>CCC(=O)O.CCC(...,7.00,3.1.1.1,0.845098,CCC(=O)OCC(COC(=O)CC)OC(=O)CC.O,CCC(=O)O.CCC(=O)OCC(CO)OC(=O)CC,0.845098,"[0.25240317, -0.45278314, -0.76844007, 0.04716...","[-0.06076221, -0.21264695, -0.05684641, 0.0163..."
1,MLLAQRRLISLGCRSKPIKTIYSSSKVLGLCTSAKMALKFKNAKRI...,brenda,Q71RI9,Nc1ccccc1C(=O)C[C@H](N)C(=O)O.O=C(O)C(=O)Cc1cc...,1.48,2.6.1.7,0.170262,Nc1ccccc1C(=O)C[C@H](N)C(=O)O.O=C(O)C(=O)Cc1cc...,N[C@@H](Cc1ccc(O)cc1)C(=O)O.O=C(O)c1cc(=O)c2cc...,0.170262,"[0.066915624, -0.15937805, -0.7076091, 0.12864...","[-0.051194582, -0.07116077, 0.16383636, 0.2113..."
2,MPIIQMNLLEGRTVEQKRNAVAAITEAVVRTLDVRPDQVRILINEL...,brenda,A2SL37,C/C(=C\C=C(/O)C(=O)O)C(=O)O>>CC(/C=C/C(=O)C(=O...,3.40,5.3.2.6,0.531479,C/C(=C\C=C(/O)C(=O)O)C(=O)O,CC(/C=C/C(=O)C(=O)O)C(=O)O,2.029384,"[0.23911352, 0.011175589, -0.35101113, 0.15938...","[-0.1772431, -0.06500952, 0.18246448, -0.05431..."
3,MKEAGILFEELVSIMEKLRSPEGCEWDRKQTHESLKPYLIEECYEL...,brenda,Q9X015,N=c1nc(O)c2ncn([C@@H]3O[C@H](COP(=O)(O)OP(=O)(...,0.69,3.6.1.9,-0.161151,N=c1nc(O)c2ncn([C@@H]3O[C@H](COP(=O)(O)OP(=O)(...,N=c1nc(O)c2ncn([C@@H]3O[C@H](COP(=O)(O)OP(=O)(...,-0.161151,"[0.078591675, -0.15656142, -0.55292135, -0.245...","[-0.09246727, -0.033019878, 0.10869139, 0.2953..."
4,MGQEKLYIEKELSWLSFNERVLQEAADKSNPLIERMRFLGIYSNNL...,sabio,P0A7B1,O=c1ccn(C2OC(COP(=O)(O)OP(=O)(O)O)C(O)C2O)c(=O...,420.00,2.7.4.1,2.623249,O=c1ccn(C2OC(COP(=O)(O)OP(=O)(O)O)C(O)C2O)c(=O...,O=c1ccn(C2OC(COP(=O)(O)OP(=O)(O)OP(=O)(O)O)C(O...,2.623249,"[0.00051207165, -0.7151457, -0.95394605, -0.15...","[-0.06592023, -0.062496405, -0.012068139, 0.06..."


In [43]:
data_kcat.rename(columns={'log10kcat_max':'label'},inplace=True)
data_kcat.head()

,Sequence,sequence_source,uniprot,Smiles,value,ec,log10_value,reactant_smiles,product_smiles,label,metabolite_features,protein_features
0,MEPGSKSVSRSDWQPEPHQRPITPLEPGPEKTPIAQPESKTLQGSN...,sabio,Q05469,CCC(=O)OCC(COC(=O)CC)OC(=O)CC.O>>CCC(=O)O.CCC(...,7.00,3.1.1.1,0.845098,CCC(=O)OCC(COC(=O)CC)OC(=O)CC.O,CCC(=O)O.CCC(=O)OCC(CO)OC(=O)CC,0.845098,"[0.25240317, -0.45278314, -0.76844007, 0.04716...","[-0.06076221, -0.21264695, -0.05684641, 0.0163..."
1,MLLAQRRLISLGCRSKPIKTIYSSSKVLGLCTSAKMALKFKNAKRI...,brenda,Q71RI9,Nc1ccccc1C(=O)C[C@H](N)C(=O)O.O=C(O)C(=O)Cc1cc...,1.48,2.6.1.7,0.170262,Nc1ccccc1C(=O)C[C@H](N)C(=O)O.O=C(O)C(=O)Cc1cc...,N[C@@H](Cc1ccc(O)cc1)C(=O)O.O=C(O)c1cc(=O)c2cc...,0.170262,"[0.066915624, -0.15937805, -0.7076091, 0.12864...","[-0.051194582, -0.07116077, 0.16383636, 0.2113..."
2,MPIIQMNLLEGRTVEQKRNAVAAITEAVVRTLDVRPDQVRILINEL...,brenda,A2SL37,C/C(=C\C=C(/O)C(=O)O)C(=O)O>>CC(/C=C/C(=O)C(=O...,3.40,5.3.2.6,0.531479,C/C(=C\C=C(/O)C(=O)O)C(=O)O,CC(/C=C/C(=O)C(=O)O)C(=O)O,2.029384,"[0.23911352, 0.011175589, -0.35101113, 0.15938...","[-0.1772431, -0.06500952, 0.18246448, -0.05431..."
3,MKEAGILFEELVSIMEKLRSPEGCEWDRKQTHESLKPYLIEECYEL...,brenda,Q9X015,N=c1nc(O)c2ncn([C@@H]3O[C@H](COP(=O)(O)OP(=O)(...,0.69,3.6.1.9,-0.161151,N=c1nc(O)c2ncn([C@@H]3O[C@H](COP(=O)(O)OP(=O)(...,N=c1nc(O)c2ncn([C@@H]3O[C@H](COP(=O)(O)OP(=O)(...,-0.161151,"[0.078591675, -0.15656142, -0.55292135, -0.245...","[-0.09246727, -0.033019878, 0.10869139, 0.2953..."
4,MGQEKLYIEKELSWLSFNERVLQEAADKSNPLIERMRFLGIYSNNL...,sabio,P0A7B1,O=c1ccn(C2OC(COP(=O)(O)OP(=O)(O)O)C(O)C2O)c(=O...,420.00,2.7.4.1,2.623249,O=c1ccn(C2OC(COP(=O)(O)OP(=O)(O)O)C(O)C2O)c(=O...,O=c1ccn(C2OC(COP(=O)(O)OP(=O)(O)OP(=O)(O)O)C(O...,2.623249,"[0.00051207165, -0.7151457, -0.95394605, -0.15...","[-0.06592023, -0.062496405, -0.012068139, 0.06..."


In [44]:
# Define dataset
class MPI_Dataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        return {
            'metabolite_features': torch.tensor(np.asarray(row['metabolite_features'], dtype=np.float32)),
            'protein_features': torch.tensor(np.asarray(row['protein_features'], dtype=np.float32)),
            'label': torch.tensor(float(row['label']), dtype=torch.float32),
        }

In [45]:
# Separate the dataset by unique proteins and drugs
unique_proteins = data_kcat['Sequence'].unique()
unique_mols = data_kcat['Smiles'].unique()
# Set the seed for reproducibility
torch.manual_seed(42)
# Function to perform a cold split
def cold_split(unique_items, test_size=0.2, val_size=0.1):
    train_items, test_items = train_test_split(unique_items, test_size=test_size, random_state=42)
    train_items, val_items = train_test_split(train_items, test_size=val_size / (1 - test_size), random_state=42)
    return train_items, val_items, test_items
# Cold split by proteins
train_proteins, val_proteins, test_proteins = cold_split(unique_proteins)
train_cold_protein = data_kcat[data_kcat['Sequence'].isin(train_proteins)]
val_cold_protein = data_kcat[data_kcat['Sequence'].isin(val_proteins)]
test_cold_protein = data_kcat[data_kcat['Sequence'].isin(test_proteins)]
# Cold split by molecules
train_mols, val_mols, test_mols = cold_split(unique_mols)
train_cold_mols = data_kcat[data_kcat['Smiles'].isin(train_mols)]
val_cold_mols = data_kcat[data_kcat['Smiles'].isin(val_mols)]
test_cold_mols = data_kcat[data_kcat['Smiles'].isin(test_mols)]

In [46]:
def df2array(df):
    X = np.array([
    np.concatenate([m, p])
    for m, p in zip(df['metabolite_features'], df['protein_features'])])
    y = df['label']
    return X, y

train_X, train_y = df2array(train_cold_protein)
val_X, val_y = df2array(val_cold_protein)
test_X, test_y = df2array(test_cold_protein)

train_X_mols, train_y_mols = df2array(train_cold_mols)
val_X_mols, val_y_mols = df2array(val_cold_mols)
test_X_mols, test_y_mols = df2array(test_cold_mols)

In [47]:
# cold protein
train_dataset = MPI_Dataset(train_cold_protein)
val_dataset = MPI_Dataset(val_cold_protein)
test_dataset = MPI_Dataset(test_cold_protein)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# cold mols
train_dataset_mols = MPI_Dataset(train_cold_mols)
val_dataset_mols = MPI_Dataset(val_cold_mols)
test_dataset_mols = MPI_Dataset(test_cold_mols)

train_loader_mols = DataLoader(train_dataset_mols, batch_size=32, shuffle=True)
val_loader_mols = DataLoader(val_dataset_mols, batch_size=32)
test_loader_mols = DataLoader(test_dataset_mols, batch_size=32)

In [48]:
# Check CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [49]:
import torch.nn as nn
import torch
import torch.nn.functional as F

# Define MLP Model for Regression
class MLPRegressor(nn.Module):
    def __init__(self, mol_input_dim, protein_input_dim, hidden_dim=128):
        super(MLPRegressor, self).__init__()

        self.mol_encoder = nn.Linear(mol_input_dim, hidden_dim)
        self.protein_encoder = nn.Linear(protein_input_dim, hidden_dim)
        
        # norm layer + Dropout
        self.layer_norm = nn.LayerNorm(hidden_dim * 2)
        self.dropout = nn.Dropout(0.3)

        # hidden layer + output layer
        self.hidden = nn.Linear(hidden_dim * 2, hidden_dim)
        self.regressor = nn.Linear(hidden_dim, 1)

        self.activation = nn.ReLU()

    def forward(self, mol_input, protein_input):
        # Reshape to (B, 1, L) for Conv1d
        mol_embedding = self.activation(self.mol_encoder(mol_input))
        protein_embedding = self.activation(self.protein_encoder(protein_input))

        # Concatenate + Normalize + Dropout
        combined = torch.cat((mol_embedding, protein_embedding), dim=-1)
        combined = self.layer_norm(combined)
        combined = self.dropout(combined)

        # Hidden → Regress
        hidden_out = self.activation(self.hidden(combined))
        output = self.regressor(hidden_out)

        return output.squeeze(1)


In [50]:
#import cold mols models
import torch
for batch in test_loader_mols:
    mol_input_dim = batch['metabolite_features'].shape[1]
    protein_input_dim = batch['protein_features'].shape[1]
    break  
mlp = MLPRegressor(mol_input_dim=mol_input_dim, protein_input_dim=protein_input_dim,hidden_dim=512)
mlp.load_state_dict(torch.load('/Users/pinchichen/2025S lab/AI drug project/Catpred/Kcat/trained_model/cold_mols/MLP model_Catpred_cold_mols.pt'))
rf = joblib.load('/Users/pinchichen/2025S lab/AI drug project/Catpred/Kcat/trained_model/cold_mols/RF model_Catpred_cold_mols.joblib')

In [51]:
#import cold protein models
for batch in test_loader:
    mol_input_dim = batch['metabolite_features'].shape[1]
    protein_input_dim = batch['protein_features'].shape[1]
    break  
mlp_2 = MLPRegressor(mol_input_dim=mol_input_dim, protein_input_dim=protein_input_dim,hidden_dim=512)
mlp_2.load_state_dict(torch.load('/Users/pinchichen/2025S lab/AI drug project/Catpred/Kcat/trained_model/cold_protein/MLP model_Catpred_cold_protein.pt'))
rf_2 = joblib.load('/Users/pinchichen/2025S lab/AI drug project/Catpred/Kcat/trained_model/cold_protein/RF model_Catpred_cold_protein.joblib')

In [52]:
mlp.eval()
test_pred_mlp, test_labels = [], []
with torch.no_grad():
    for batch in test_loader_mols:
        mol_features = batch['metabolite_features'].to(device)
        protein_features = batch['protein_features'].to(device)
        labels = batch['label'].to(device)

        outputs = mlp(mol_features, protein_features)
        test_pred_mlp.extend(outputs.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

In [53]:
test_pred_rf = rf.predict(test_X_mols)
pred_final = (test_pred_mlp + test_pred_rf) / 2.0

In [54]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error, explained_variance_score
from scipy.stats import pearsonr
def evaluate_model(predictions, labels):
    mse = mean_squared_error(labels, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(labels, predictions)
    r2 = r2_score(labels, predictions)
    pearson_corr, _ = pearsonr(labels, predictions)
    medae = median_absolute_error(labels, predictions)
    evs = explained_variance_score(labels, predictions)

    return mse, rmse, mae, r2, pearson_corr, medae, evs

In [55]:
mlp.eval()
train_pred_mlp, train_labels = [], []
with torch.no_grad():
    for batch in train_loader_mols:
        mol_features = batch['metabolite_features'].to(device)
        protein_features = batch['protein_features'].to(device)
        labels = batch['label'].to(device)

        outputs = mlp(mol_features, protein_features)
        train_pred_mlp.extend(outputs.cpu().numpy())
        train_labels.extend(labels.cpu().numpy())

train_pred_rf = rf.predict(train_X_mols)
train_pred_final = (train_pred_mlp + train_pred_rf) / 2.0
train_mse, train_rmse, train_mae, train_r2, train_pearson_corr, train_median_ae, train_explained_var = evaluate_model(train_pred_final, train_labels)


In [56]:
mlp.eval()
val_pred_mlp, val_labels = [], []
with torch.no_grad():
    for batch in val_loader_mols:
        mol_features = batch['metabolite_features'].to(device)
        protein_features = batch['protein_features'].to(device)
        labels = batch['label'].to(device)

        outputs = mlp(mol_features, protein_features)
        val_pred_mlp.extend(outputs.cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

val_pred_rf = rf.predict(val_X_mols)
val_pred_final = (val_pred_mlp + val_pred_rf) / 2.0
val_mse, val_rmse, val_mae, val_r2, val_pearson_corr, val_median_ae, val_explained_var = evaluate_model(val_pred_final, val_labels)

In [57]:
#Choose the best model and test its performance

#best_model = regressor.best_estimator_
#best_params = regressor.best_params_
#print('Best Hyperparameters:',best_params)

# Test
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error, explained_variance_score
from scipy.stats import pearsonr
test_mse = mean_squared_error(test_y_mols, pred_final)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(test_y_mols, pred_final)
test_r2 = r2_score(test_y_mols, pred_final)
test_pearson_corr, _ = pearsonr(test_y_mols, pred_final)
test_median_ae = median_absolute_error(test_y_mols, pred_final)
test_explained_var = explained_variance_score(test_y_mols, pred_final)


print(f"Test MSE: {test_mse:.4f}, MAE: {test_mae:.4f}, R-square: {test_r2:.4f}")

Test MSE: 1.4645, MAE: 0.8652, R-square: 0.4830


In [58]:
# append the performance to the csv file
df = {
    'Model':['ensemble', 'ensemble', 'ensemble'],
    'Dataset':['Train', 'Validation', 'Test'],
    'MSE':[train_mse, val_mse, test_mse],
    'RMSE':[train_rmse, val_rmse, test_rmse],
    'MAE':[train_mae, val_mae, test_mae],
    'R2':[train_r2, val_r2, test_r2],
    'Pearson':[train_pearson_corr, val_pearson_corr, test_pearson_corr],
    'Median_AE':[train_median_ae, val_median_ae, test_median_ae],
    'Explained_VAR':[train_explained_var, val_explained_var, test_explained_var],
    'Dataspliting Mode':['cold mols', 'cold mols', 'cold mols']
}
df = pd.DataFrame(df)

df.to_csv('/Users/pinchichen/2025S lab/AI drug project/Catpred/Kcat/model performance metrics_Catpred_Kcat.csv', mode='a', header=False)

In [59]:
mlp_2.eval()
train_pred_mlp, train_labels = [], []
with torch.no_grad():
    for batch in train_loader:
        mol_features = batch['metabolite_features'].to(device)
        protein_features = batch['protein_features'].to(device)
        labels = batch['label'].to(device)

        outputs = mlp_2(mol_features, protein_features)
        train_pred_mlp.extend(outputs.cpu().numpy())
        train_labels.extend(labels.cpu().numpy())

train_pred_rf = rf_2.predict(train_X)
train_pred_final = (train_pred_mlp + train_pred_rf) / 2.0
train_mse, train_rmse, train_mae, train_r2, train_pearson_corr, train_median_ae, train_explained_var = evaluate_model(train_pred_final, train_labels)


In [60]:
mlp_2.eval()
val_pred_mlp, val_labels = [], []
with torch.no_grad():
    for batch in val_loader:
        mol_features = batch['metabolite_features'].to(device)
        protein_features = batch['protein_features'].to(device)
        labels = batch['label'].to(device)

        outputs = mlp_2(mol_features, protein_features)
        val_pred_mlp.extend(outputs.cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

val_pred_rf = rf_2.predict(val_X)
val_pred_final = (val_pred_mlp + val_pred_rf) / 2.0
val_mse, val_rmse, val_mae, val_r2, val_pearson_corr, val_median_ae, val_explained_var = evaluate_model(val_pred_final, val_labels)

In [61]:
mlp_2.eval()
test_pred_mlp, test_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        mol_features = batch['metabolite_features'].to(device)
        protein_features = batch['protein_features'].to(device)
        labels = batch['label'].to(device)

        outputs = mlp_2(mol_features, protein_features)
        test_pred_mlp.extend(outputs.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

In [62]:
test_pred_rf = rf_2.predict(test_X)
pred_final = (test_pred_mlp + test_pred_rf) / 2.0

In [63]:
test_mse = mean_squared_error(test_y, pred_final)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(test_y, pred_final)
test_r2 = r2_score(test_y, pred_final)
test_pearson_corr, _ = pearsonr(test_y, pred_final)
test_median_ae = median_absolute_error(test_y, pred_final)
test_explained_var = explained_variance_score(test_y, pred_final)

In [64]:
# append the performance to the csv file
df = {
    'Model':['ensemble', 'ensemble', 'ensemble'],
    'Dataset':['Train', 'Validation', 'Test'],
    'MSE':[train_mse, val_mse, test_mse],
    'RMSE':[train_rmse, val_rmse, test_rmse],
    'MAE':[train_mae, val_mae, test_mae],
    'R2':[train_r2, val_r2, test_r2],
    'Pearson':[train_pearson_corr, val_pearson_corr, test_pearson_corr],
    'Median_AE':[train_median_ae, val_median_ae, test_median_ae],
    'Explained_VAR':[train_explained_var, val_explained_var, test_explained_var],
    'Dataspliting Mode':['cold protein', 'cold protein', 'cold protein']
}
df = pd.DataFrame(df)

df.to_csv('/Users/pinchichen/2025S lab/AI drug project/Catpred/Kcat/model performance metrics_Catpred_Kcat.csv', mode='a', header=False)